In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import pandas as pd

cudnn.benchmark = True
plt.ion()   # interactive mode

In [5]:
mapping_file = "/home/ubuntu/data/HAR_datasets/GTEA/labels/mapping.txt"
file_ptr = open(mapping_file, 'r')
actions = file_ptr.read().split('\n')[:-1]
file_ptr.close()
actions_dict = dict()
for a in actions:
    actions_dict[a.split()[1]] = int(a.split()[0])
actions_dict

{'background_action': 0,
 'close_chocolate': 1,
 'close_coffee': 2,
 'close_honey': 3,
 'close_jam': 4,
 'close_ketchup': 5,
 'close_mayonnaise': 6,
 'close_mustard': 7,
 'close_peanut': 8,
 'close_sugar': 9,
 'close_water': 10,
 'fold_bread': 11,
 'open_cheese': 12,
 'open_chocolate': 13,
 'open_coffee': 14,
 'open_honey': 15,
 'open_jam': 16,
 'open_ketchup': 17,
 'open_mayonnaise': 18,
 'open_mustard': 19,
 'open_peanut': 20,
 'open_sugar': 21,
 'open_tea': 22,
 'open_water': 23,
 'pour_chocolate_bread': 24,
 'pour_coffee_spoon_cup': 25,
 'pour_honey_bread': 26,
 'pour_honey_cup': 27,
 'pour_ketchup_hotdog_bread': 28,
 'pour_mayonnaise_cheese_bread': 29,
 'pour_mustard_cheese_bread': 30,
 'pour_mustard_hotdog_bread': 31,
 'pour_sugar_spoon_cup': 32,
 'pour_water_cup': 33,
 'put_bread_bread': 34,
 'put_bread_cheese_bread': 35,
 'put_cheese_bread': 36,
 'put_chocolate': 37,
 'put_coffee': 38,
 'put_honey': 39,
 'put_hotdog_bread': 40,
 'put_jam': 41,
 'put_ketchup': 42,
 'put_mayonnai

In [ ]:
df = pd.read_csv("/home/ubuntu/local/Resnet50/csv/S1_Cheese_C1.csv")
sorted(os.listdir("/home/ubuntu/local/ssd_colab/GTEA_feats/one_image_per_frame/S1_Cheese_C1"))
df["frame_path"] = sorted(os.listdir("/home/ubuntu/local/ssd_colab/GTEA_feats/one_image_per_frame/S1_Cheese_C1"))
df["map"] = df["action"].map(actions_dict)
df = 

### Create the csv files for noun csv

In [3]:
df = pd.read_csv("/home/ubuntu/local/Resnet50/csv/S1_Cheese_C1.csv")
sorted(os.listdir("/home/ubuntu/local/ssd_colab/GTEA_feats/one_image_per_frame/S1_Cheese_C1"))
df["frame_path"] = sorted(os.listdir("/home/ubuntu/local/ssd_colab/GTEA_feats/one_image_per_frame/S1_Cheese_C1"))
df["map"] = df["action"].map(actions_dict)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/local/Resnet50/csv/S1_Cheese_C1.csv'

In [ ]:
class_names

In [ ]:
def get_mean(norm_value=255):
    # mean of imagenet
    return [
        123.675 / norm_value, 116.28 / norm_value,
        103.53 / norm_value
    ]


def get_std(norm_value=255):
    # std of imagenet
    return [
        58.395 / norm_value, 57.12 / norm_value,
        57.375 / norm_value
    ]


In [ ]:
get_mean()

In [ ]:
# Read image
# orig = cv2.imread("/home/ubuntu/local/ssd_colab/GTEA/S1_Cheese_C1/0000000017.png_hand_2.png")

# Convert image to RGB from BGR (another way is to use "image = image[:, :, ::-1]" code)
# orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

# Resize image to 224x224 size
# image = cv2.resize(orig, (224, 224)).reshape(-1, 224, 224, 3)

# We need to preprocess imageto fulfill ResNet50 requirements

image = PIL.Image.open("/home/ubuntu/local/ssd_colab/GTEA/S1_Cheese_C1/0000000017.png_hand_2.png")
spatial_transform = []
spatial_transform.append(transforms.Resize(256))
spatial_transform.append(transforms.CenterCrop(256))
spatial_transform.append(transforms.ToTensor())
spatial_transform.append(transforms.Normalize(mean=get_mean(), std=get_std()))

transform = transforms.Compose(spatial_transform)

image = transform(image)

# Extracting our features
features = resnet50.predict(image)

features.shape

## Train and evaluate model 

In [ ]:
model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 73)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
import dataset
train_data =dataset.GTEADataset(dataset_dir="/home/ubuntu/local/ssd_colab/GTEA_feats/one_image_per_frame", csv_dir="/home/ubuntu/local/Resnet50/csv/")
train_loader = dataset.DataLoader(train_data, batch_size=1, collate_fn=dataset.collate_fn)

for data in train_loader:
    print(data["images"].shape)

### Visualize features

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.utils as utils
import torch.nn as nn
from PIL import Image

# Load pre-trained ResNet-50 model
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = resnet.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
resnet.fc = nn.Linear(num_ftrs, 73)

# Load state dict 
state_dict = torch.load('/home/ubuntu/local/Resnet50/checkpoints/best_model_params_64.pt')


# Load state dict into the model
resnet.load_state_dict(state_dict)
# resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

# Set the model to evaluation mode
resnet.eval()


# Define a transformation to preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess an example image
image = Image.open("/home/ubuntu/local/Resnet50/GTEA/train/S1_Cheese_C1_0000000177.png")
image = transform(image)
image = image.unsqueeze(0)  # Add batch dimension
print("image", image.shape)
# Forward pass through ResNet-50
features = resnet.conv1(image)
features = resnet.bn1(features)
features = resnet.relu(features)
features = resnet.maxpool(features)
features = resnet.layer1(features)
features = resnet.layer2(features)
features = resnet.layer3(features)
features = resnet.layer4(features)
print("FEATURES", features.shape)
features = resnet.avgpool(features)

# features = resnet(image)
print("FEATURES", features.shape)
output = resnet(image)
print("OUT", output.shape)

# Select a subset of channels for visualization
subset_channels = [2047, 1500, 2000]

# Transpose the dimensions of features tensor
# features = features.permute(0, 2, 3, 1)

# Visualize the selected channels individually
# for channel in subset_channels:
#     grid = utils.make_grid(features[..., channel], normalize=True, scale_each=True)
#     plt.imshow(grid.permute(1, 2, 0))
#     plt.axis('off')
#     plt.show()
print(torch.max(output,1))
output

# ------------------------------------
#### Noun recognition using Resnet50 model

In [142]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.utils as utils
import torch.nn as nn
from PIL import Image
import pandas as pd

# Load pre-trained ResNet-50 model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = resnet.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
resnet.fc = nn.Linear(num_ftrs, 73)

# Load state dict 
state_dict = torch.load('/home/ubuntu/local/Resnet50/checkpoints/best_model_params_noun2_64.pt')


# Load state dict into the model
resnet.load_state_dict(state_dict)
# resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

# Set the model to evaluation mode
resnet.eval()


# Define a transformation to preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
annots = pd.read_csv("/home/ubuntu/local/Resnet50/csv/noun/train.csv")

# Load and preprocess an example image
image = Image.open("/home/ubuntu/local/Resnet50/GTEA/train/S1_Cheese_C1_0000000106.png")
image = transform(image)
image = image.unsqueeze(0)  # Add batch dimension
label = annots.loc[annots["frame_path"]=="S1_Cheese_C1_0000000177.png"].label.item()
# Forward pass through ResNet-50
output = resnet(image)
_, pred = torch.max(output,1)

print(label, pred.item())

4 4


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.utils as utils
import torch.nn as nn
from PIL import Image
import pandas as pd

# Load pre-trained ResNet-50 model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = resnet.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
resnet.fc = nn.Linear(num_ftrs, 73)

# Load state dict 
state_dict = torch.load('/home/ubuntu/local/Resnet50/checkpoints/best_model_params_noun2_64.pt')

# Define a transformation to preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load state dict into the model
resnet.load_state_dict(state_dict)
# resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.to(device)

# Set the model to evaluation mode
resnet.eval()

annots = pd.read_csv("/home/ubuntu/local/Resnet50/csv/noun/train.csv")

image = sorted(os.listdir("/home/ubuntu/local/Resnet50/GTEA/train"))
correct = 0 
preds = []
labels = []
results = pd.DataFrame(columns=['frame', 'pred', 'gt'])

for img in image[:10]:
    image = Image.open("/home/ubuntu/local/Resnet50/GTEA/train/"+img)
    image = transform(image).to(device)
    image = image.unsqueeze(0)  # Add batch dimension
    output = resnet(image)
    label = annots.loc[annots["frame_path"]==img].label.item()
    labels.append(label)
    _, pred = torch.max(output,1)
    preds.append(pred.item())
    # results = results.append({'frame' : img, 'pred' : pred.item(), 'gt' : label.item()},
    #     ignore_index = True)
    new_row = {'frame' : img, 'pred' : pred.item(), 'gt' : label}
    results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)



count = sum(x == y for x, y in zip(preds, labels))
print(count)
results

# ------------------------------

# Analysis of the results

In [6]:
import pandas as pd

Mapping noun to int

In [7]:
mapping_noun_file = "/home/ubuntu/data/hierarchical_train/data/Nmapping.txt"
file_ptr = open(mapping_noun_file, 'r')
noun_actions = file_ptr.read().split('\n')
file_ptr.close()
noun_actions_dict = dict()
for a in noun_actions:
    noun_actions_dict[a.split()[1]] = int(a.split()[0])

num_classes = len(noun_actions_dict)
noun_actions_dict

{'action': 0,
 'bread': 1,
 'bread_bread': 2,
 'bread_cheese_bread': 3,
 'cheese': 4,
 'cheese_bread': 5,
 'chocolate': 6,
 'chocolate_bread': 7,
 'coffee': 8,
 'coffee_spoon': 9,
 'coffee_spoon_cup': 10,
 'cup': 11,
 'honey': 12,
 'honey_bread': 13,
 'honey_cup': 14,
 'hotdog': 15,
 'hotdog_bread': 16,
 'jam': 17,
 'jam_spoon': 18,
 'jam_spoon_bread': 19,
 'ketchup': 20,
 'ketchup_hotdog_bread': 21,
 'mayonnaise': 22,
 'mayonnaise_cheese_bread': 23,
 'mustard': 24,
 'mustard_cheese_bread': 25,
 'mustard_hotdog_bread': 26,
 'peanut': 27,
 'peanut_spoon': 28,
 'peanut_spoon_bread': 29,
 'spoon': 30,
 'spoon_cup': 31,
 'sugar': 32,
 'sugar_spoon': 33,
 'sugar_spoon_cup': 34,
 'tea': 35,
 'tea_cup': 36,
 'water': 37,
 'water_cup': 38}

In [15]:
mapping_verb_file = "/home/ubuntu/data/hierarchical_train/data/Vmapping.txt"
file_ptr = open(mapping_verb_file, 'r')
verb_actions = file_ptr.read().split('\n')
file_ptr.close()
verb_actions_dict = dict()
for a in verb_actions:
    verb_actions_dict[a.split()[1]] = int(a.split()[0])

num_classes = len(verb_actions_dict)
verb_actions_dict

{'take': 0,
 'open': 1,
 'pour': 2,
 'close': 3,
 'shake': 4,
 'scoop': 5,
 'stir': 6,
 'put': 7,
 'fold': 8,
 'spread': 9,
 'background': 10}

In [8]:
split = 1

In [5]:
results = pd.read_csv("/home/ubuntu/local/Resnet50/results/weighted/noun/N_split"+str(split)+".csv")
results = results[["frame", "pred", "gt"]]
results['pred'] = results['pred'].map({v: k for k, v in noun_actions_dict.items()})
results['gt'] = results['gt'].map({v: k for k, v in noun_actions_dict.items()})
results


,frame,pred,gt
0,S1_Cheese_C1_0000000001.png,spoon,action
1,S1_Cheese_C1_0000000002.png,spoon,action
2,S1_Cheese_C1_0000000003.png,spoon,action
3,S1_Cheese_C1_0000000004.png,spoon,action
4,S1_Cheese_C1_0000000005.png,spoon,action
...,...,...,...
9105,S1_Tea_C1_0000002005.png,water,action
9106,S1_Tea_C1_0000002006.png,action,action
9107,S1_Tea_C1_0000002007.png,action,action
9108,S1_Tea_C1_0000002008.png,action,action


The correctly labelled frames

In [6]:
results.loc[results["pred"]==results["gt"]]

,frame,pred,gt
10,S1_Cheese_C1_0000000011.png,action,action
17,S1_Cheese_C1_0000000018.png,action,action
21,S1_Cheese_C1_0000000022.png,action,action
22,S1_Cheese_C1_0000000023.png,action,action
24,S1_Cheese_C1_0000000025.png,action,action
...,...,...,...
9104,S1_Tea_C1_0000002004.png,action,action
9106,S1_Tea_C1_0000002006.png,action,action
9107,S1_Tea_C1_0000002007.png,action,action
9108,S1_Tea_C1_0000002008.png,action,action


In [7]:
5636/9110*100

61.86608122941822

In [16]:
# for "noun" and "verb"
for split in range(1,5):
    vids = sorted(os.listdir("/home/ubuntu/data/MS-TCN2-master/results/gtea/split_"+str(split)))
    for vid in vids:
        print(vid)
        results = pd.read_csv("/home/ubuntu/local/Resnet50/results/weighted/verb/V_split"+str(split)+".csv")
        results = results[["frame", "pred", "gt"]]
        results['pred'] = results['pred'].map({v: k for k, v in verb_actions_dict.items()})
        results['gt'] = results['gt'].map({v: k for k, v in verb_actions_dict.items()})
        filtered_df = results[results['frame'].squeeze().str.contains(vid)].reset_index(drop=True)
        filtered_df.to_csv("/home/ubuntu/local/Resnet50/results/weighted/verb/split"+str(split)+"/noun_vids/"+vid+".csv", index=False)

S1_Cheese_C1
S1_CofHoney_C1
S1_Coffee_C1
S1_Hotdog_C1
S1_Pealate_C1
S1_Peanut_C1
S1_Tea_C1
S2_Cheese_C1
S2_CofHoney_C1
S2_Coffee_C1
S2_Hotdog_C1
S2_Pealate_C1
S2_Peanut_C1
S2_Tea_C1
S3_Cheese_C1
S3_CofHoney_C1
S3_Coffee_C1
S3_Hotdog_C1
S3_Pealate_C1
S3_Peanut_C1
S3_Tea_C1
S4_Cheese_C1
S4_CofHoney_C1
S4_Coffee_C1
S4_Hotdog_C1
S4_Pealate_C1
S4_Peanut_C1
S4_Tea_C1


## Join verb and noun

In [36]:
def full_results(split, video):
    verb = pd.read_csv("/home/ubuntu/local/Resnet50/results/weighted/verb/split"+str(split)+"/noun_vids/"+video+".csv")
    noun = pd.read_csv("/home/ubuntu/local/Resnet50/results/weighted/noun/split"+str(split)+"/noun_vids/"+video+".csv")
    full = pd.merge(verb,noun, on="frame")
    full["gt"] = full["gt_x"] + "_" + full["gt_y"]
    full["prediction"] = full["pred_x"] + "_" + full["pred_y"]
    full = full.drop(["gt_x", "gt_y"], axis=1)
    full = full.rename(columns={"pred_x":"verb", "pred_y":"noun"})
    full = full[["verb", "noun", "gt", "prediction"]]
    return full

In [35]:
full_results(1,"S1_Cheese_C1")

,verb,noun,gt,prediction
0,background,spoon,background_action,background_spoon
1,background,spoon,background_action,background_spoon
2,background,spoon,background_action,background_spoon
3,background,spoon,background_action,background_spoon
4,background,spoon,background_action,background_spoon
...,...,...,...,...
938,background,spoon,background_action,background_spoon
939,background,spoon,background_action,background_spoon
940,background,spoon,background_action,background_spoon
941,background,spoon,background_action,background_spoon


### Confusion matrix

In [12]:
results

,frame,pred,gt
0,S1_Cheese_C1_0000000001.png,spoon,action
1,S1_Cheese_C1_0000000002.png,spoon,action
2,S1_Cheese_C1_0000000003.png,spoon,action
3,S1_Cheese_C1_0000000004.png,spoon,action
4,S1_Cheese_C1_0000000005.png,spoon,action
...,...,...,...
9105,S1_Tea_C1_0000002005.png,water,action
9106,S1_Tea_C1_0000002006.png,action,action
9107,S1_Tea_C1_0000002007.png,action,action
9108,S1_Tea_C1_0000002008.png,action,action


In [13]:
## sklearn conf_mat function 
from sklearn import metrics
gt_n_array = results["gt"].values.tolist()
pred_n_array = results["pred"].values.tolist()

conf_mat_v = metrics.confusion_matrix(gt_n_array, pred_n_array)
# conf_mat = metrics.multilabel_confusion_matrix(gt_array, pred_array, labels=list(range(0,73)))
conf_mat_v

array([[1161,   92,  229, ...,   13,   46,   18],
       [  39,   89,    5, ...,    0,    0,    0],
       [   9,    9,  149, ...,    0,    0,    0],
       ...,
       [ 493,    0,    0, ...,   84,    0,   50],
       [  14,    3,    0, ...,    0,  224,    7],
       [  13,    0,    0, ...,    0,   13,  444]])

In [14]:
noun_list = list(noun_actions_dict.keys())
noun_list

['action',
 'bread',
 'bread_bread',
 'bread_cheese_bread',
 'cheese',
 'cheese_bread',
 'chocolate',
 'chocolate_bread',
 'coffee',
 'coffee_spoon',
 'coffee_spoon_cup',
 'cup',
 'honey',
 'honey_bread',
 'honey_cup',
 'hotdog',
 'hotdog_bread',
 'jam',
 'jam_spoon',
 'jam_spoon_bread',
 'ketchup',
 'ketchup_hotdog_bread',
 'mayonnaise',
 'mayonnaise_cheese_bread',
 'mustard',
 'mustard_cheese_bread',
 'mustard_hotdog_bread',
 'peanut',
 'peanut_spoon',
 'peanut_spoon_bread',
 'spoon',
 'spoon_cup',
 'sugar',
 'sugar_spoon',
 'sugar_spoon_cup',
 'tea',
 'tea_cup',
 'water',
 'water_cup']

In [15]:
import numpy as np
c_matrix = np.zeros((len(noun_actions_dict),len(noun_actions_dict)))
for gt, pred in zip(gt_n_array, pred_n_array):
    c_matrix[noun_list.index(gt)][noun_list.index(pred)]+=1
c_matrix

array([[1161.,   92.,  229., ...,   13.,   46.,   18.],
       [  39.,   89.,    5., ...,    0.,    0.,    0.],
       [   9.,    9.,  149., ...,    0.,    0.,    0.],
       ...,
       [ 493.,    0.,    0., ...,   84.,    0.,   50.],
       [  14.,    3.,    0., ...,    0.,  224.,    7.],
       [  13.,    0.,    0., ...,    0.,   13.,  444.]])

From the confusion matrix, get the classwise accuracy 


In [16]:
acc = []
for i in range(len(c_matrix)):
    acc.append(c_matrix[i][i]/sum(c_matrix[i])*100)
acc

/tmp/ipykernel_43088/536053248.py:3: RuntimeWarning: invalid value encountered in scalar divide
  acc.append(c_matrix[i][i]/sum(c_matrix[i])*100)


[47.95539033457249,
 41.2037037037037,
 81.86813186813187,
 14.285714285714285,
 50.29239766081871,
 78.78787878787878,
 91.13924050632912,
 90.47619047619048,
 72.32472324723247,
 89.375,
 57.74647887323944,
 nan,
 47.95918367346938,
 78.88446215139442,
 94.68085106382979,
 14.634146341463413,
 65.0,
 70.2127659574468,
 89.28571428571429,
 41.333333333333336,
 51.11111111111111,
 37.83783783783784,
 74.15730337078652,
 49.45054945054945,
 66.28571428571428,
 96.7741935483871,
 79.8507462686567,
 60.410557184750736,
 85.71428571428571,
 75.0455373406193,
 60.66176470588235,
 76.52370203160271,
 88.25910931174089,
 80.0,
 63.793103448275865,
 88.10408921933085,
 13.003095975232199,
 84.84848484848484,
 93.47368421052632]

Verb dataset

In [89]:
def pred_df(split, file):
    df = pd.read_csv("/home/ubuntu/data/hierarchical_train/results/gtea-verb/split_" + str(split) + "/" + file)
    df.columns = ["vpred"]
    df = df.drop([0,1,2,3])
    df = df.reset_index().drop(columns=["index"])
    # df = df.replace({"prediction":actions_dict})
    return df
pred_df(1,"S1_Cheese_C1")

,vpred
0,background
1,background
2,background
3,background
4,background
...,...
938,background
939,background
940,background
941,background


In [90]:
print(split)
full_verb = pd.DataFrame(columns=["vpred"])

list_vids = sorted(os.listdir('/home/ubuntu/data/hierarchical_train/results/gtea-verb/split_' + str(split)))
for vid in list_vids:
    df = pred_df(split, vid)
    full_verb = pd.concat([full_verb, df], ignore_index=True)
full_verb

1


,vpred
0,background
1,background
2,background
3,background
4,background
...,...
9105,background
9106,background
9107,background
9108,background


GT data

In [91]:
def gt_df(file):
    df = pd.read_csv("/home/ubuntu/data/HAR_datasets/GTEA/labels/final_labels_txt/" + file + ".txt", header=None)
    df.columns = ["gt"]
    # df = df.drop([0,1,2,3])
    df = df.reset_index().drop(columns=["index"])
    # df = df.replace({"prediction":actions_dict})
    return df

In [92]:
list_vids = sorted(os.listdir('/home/ubuntu/data/hierarchical_train/results/gtea-verb/split_' + str(split)))
print(list_vids)
full_gt = pd.DataFrame(columns=["gt"])
for vid in list_vids:
    df = gt_df(vid)
    full_gt = pd.concat([full_gt, df], ignore_index=True)
full_gt


['S1_Cheese_C1', 'S1_CofHoney_C1', 'S1_Coffee_C1', 'S1_Hotdog_C1', 'S1_Pealate_C1', 'S1_Peanut_C1', 'S1_Tea_C1']


,gt
0,background_action
1,background_action
2,background_action
3,background_action
4,background_action
...,...
9105,background_action
9106,background_action
9107,background_action
9108,background_action


In [93]:
final_df = results[["frame", "pred", "gt"]]
final_df = pd.concat([final_df, full_verb], ignore_index=True, axis=1)
final_df

,0,1,2,3
0,S1_Cheese_C1_0000000001.png,action,action,background
1,S1_Cheese_C1_0000000002.png,action,action,background
2,S1_Cheese_C1_0000000003.png,action,action,background
3,S1_Cheese_C1_0000000004.png,action,action,background
4,S1_Cheese_C1_0000000005.png,action,action,background
...,...,...,...,...
9105,S1_Tea_C1_0000002005.png,action,action,background
9106,S1_Tea_C1_0000002006.png,action,action,background
9107,S1_Tea_C1_0000002007.png,action,action,background
9108,S1_Tea_C1_0000002008.png,action,action,background


In [94]:
final_df.loc[final_df[1]!=final_df[2]]

,0,1,2,3
8,S1_Cheese_C1_0000000009.png,water_cup,action,background
18,S1_Cheese_C1_0000000019.png,bread,action,background
19,S1_Cheese_C1_0000000020.png,bread,action,background
53,S1_Cheese_C1_0000000054.png,spoon,action,background
57,S1_Cheese_C1_0000000058.png,water,action,background
...,...,...,...,...
9026,S1_Tea_C1_0000001926.png,tea_cup,spoon_cup,stir
9027,S1_Tea_C1_0000001927.png,action,spoon_cup,stir
9028,S1_Tea_C1_0000001928.png,action,spoon_cup,stir
9030,S1_Tea_C1_0000001930.png,action,spoon_cup,stir


In [95]:
final_df = pd.concat([final_df, full_gt], ignore_index=True, axis=1)
final_df.columns = [["frame", "noun", "gt_noun", "verb", "GT"]]
final_df

,frame,noun,gt_noun,verb,GT
0,S1_Cheese_C1_0000000001.png,action,action,background,background_action
1,S1_Cheese_C1_0000000002.png,action,action,background,background_action
2,S1_Cheese_C1_0000000003.png,action,action,background,background_action
3,S1_Cheese_C1_0000000004.png,action,action,background,background_action
4,S1_Cheese_C1_0000000005.png,action,action,background,background_action
...,...,...,...,...,...
9105,S1_Tea_C1_0000002005.png,action,action,background,background_action
9106,S1_Tea_C1_0000002006.png,action,action,background,background_action
9107,S1_Tea_C1_0000002007.png,action,action,background,background_action
9108,S1_Tea_C1_0000002008.png,action,action,background,background_action


In [96]:
final_df["pred"] = final_df[['verb', 'noun']].agg('_'.join, axis=1).reset_index(drop=True)
final_df

,frame,noun,gt_noun,verb,GT,pred
0,S1_Cheese_C1_0000000001.png,action,action,background,background_action,background_action
1,S1_Cheese_C1_0000000002.png,action,action,background,background_action,background_action
2,S1_Cheese_C1_0000000003.png,action,action,background,background_action,background_action
3,S1_Cheese_C1_0000000004.png,action,action,background,background_action,background_action
4,S1_Cheese_C1_0000000005.png,action,action,background,background_action,background_action
...,...,...,...,...,...,...
9105,S1_Tea_C1_0000002005.png,action,action,background,background_action,background_action
9106,S1_Tea_C1_0000002006.png,action,action,background,background_action,background_action
9107,S1_Tea_C1_0000002007.png,action,action,background,background_action,background_action
9108,S1_Tea_C1_0000002008.png,action,action,background,background_action,background_action


In [97]:
gts = final_df["GT"].reset_index(drop=True)
type(gts)
n = final_df.iloc[:,5]
ser = final_df.iloc[:,4]

(ser == n).sum()


4526

Create csv_file for each video to compute edit score and F1 

In [98]:
list_vids

['S1_Cheese_C1',
 'S1_CofHoney_C1',
 'S1_Coffee_C1',
 'S1_Hotdog_C1',
 'S1_Pealate_C1',
 'S1_Peanut_C1',
 'S1_Tea_C1']

In [99]:
for vid in list_vids:
    filtered_df = final_df[final_df['frame'].squeeze().str.contains(vid)].reset_index(drop=True)
    filtered_df.to_csv("/home/ubuntu/local/Resnet50/results/noun/split"+str(split)+"/vids/"+vid+".csv", index=False)

Edit score  and F1 score

In [17]:
import numpy as np
import argparse
import wandb
import os
import pandas as pd

def read_file(path):
    with open(path, 'r') as f:
        content = f.read()
        f.close()
    return content


def get_labels_start_end_time(frame_wise_labels, bg_class=["action"]):
    labels = []
    starts = []
    ends = []
    last_label = frame_wise_labels[0]
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i)
            last_label = frame_wise_labels[i]
    if last_label not in bg_class:
        ends.append(i)
    return labels, starts, ends


def levenstein(p, y, norm=False):
    m_row = len(p)
    n_col = len(y)
    D = np.zeros([m_row+1, n_col+1], float)
    for i in range(m_row+1):
        D[i, 0] = i
    for i in range(n_col+1):
        D[0, i] = i

    for j in range(1, n_col+1):
        for i in range(1, m_row+1):
            if y[j-1] == p[i-1]:
                D[i, j] = D[i-1, j-1]
            else:
                D[i, j] = min(D[i-1, j] + 1,
                              D[i, j-1] + 1,
                              D[i-1, j-1] + 1)

    if norm:
        score = (1 - D[-1, -1]/max(m_row, n_col)) * 100
    else:
        score = D[-1, -1]

    return score


def edit_score(recognized, ground_truth, norm=True, bg_class=["background_action"]):
    P, _, _ = get_labels_start_end_time(recognized, bg_class)
    Y, _, _ = get_labels_start_end_time(ground_truth, bg_class)
    return levenstein(P, Y, norm)


def f_score(recognized, ground_truth, overlap, bg_class=["background_action"]):
    p_label, p_start, p_end = get_labels_start_end_time(recognized, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(ground_truth, bg_class)

    tp = 0
    fp = 0

    hits = np.zeros(len(y_label))

    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
        IoU = (1.0*intersection / union)*([p_label[j] == y_label[x] for x in range(len(y_label))])
        # Get the best scoring segment
        idx = np.array(IoU).argmax()

        if IoU[idx] >= overlap and not hits[idx]:
            tp += 1
            hits[idx] = 1
        else:
            fp += 1
    fn = len(y_label) - sum(hits)
    return float(tp), float(fp), float(fn)




For all the videos

In [3]:
correct = 0
total = 0
edit = 0
overlap = [.1, .25, .5]
tp, fp, fn = np.zeros(3), np.zeros(3), np.zeros(3)

print("SPLIT ", split)
correct = 0
total = 0
edit = 0
overlap = [.1, .25, .5]
tp, fp, fn = np.zeros(3), np.zeros(3), np.zeros(3)

for vid in list_vids:
        df = pd.read_csv("/home/ubuntu/local/Resnet50/results/noun/split"+str(split)+"/vids/"+vid+".csv")
        gt_list = df["GT"].squeeze().tolist()
        pred_list = df["pred"].squeeze().tolist()
        edit += edit_score(pred_list, gt_list)

        for i in range(len(gt_list)):
                total+=1
                if gt_list[i]==pred_list[i]:
                        correct+=1

        for s in range(len(overlap)):
                        tp1, fp1, fn1 = f_score(pred_list, gt_list, overlap[s])
                        tp[s] += tp1
                        fp[s] += fp1
                        fn[s] += fn1

print("ACCURACY", correct/total * 100)

print("EDIT", edit/7)

for s in range(len(overlap)):
        precision = tp[s] / float(tp[s]+fp[s])
        recall = tp[s] / float(tp[s]+fn[s])

        f1 = 2.0 * (precision*recall) / (precision+recall)

        f1 = np.nan_to_num(f1)*100
        print('F1@%0.2f: %.4f' % (overlap[s], f1))

        



SPLIT  1


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/local/Resnet50/results/noun/split1/vids/S1_Cheese_C1.csv'

In [38]:
ACC = []
EDIT = []
F1_10 = []
F1_25 = []
F1_50 = []
for split in range(1, 5):
        print("SPLIT ", split)
        list_vids = os.listdir('/home/ubuntu/data/hierarchical_train/results/verb/split_' + str(split))
        correct = 0
        total = 0
        edit = 0
        overlap = [.1, .25, .5]
        tp, fp, fn = np.zeros(3), np.zeros(3), np.zeros(3)


        for vid in list_vids:
                df = full_results(split, vid)
                gt_list = df["gt"].squeeze().tolist()
                pred_list = df["prediction"].squeeze().tolist()
                edit += edit_score(pred_list, gt_list)

                for i in range(len(gt_list)):
                        total+=1
                        if gt_list[i]==pred_list[i]:
                                correct+=1

                for s in range(len(overlap)):
                                tp1, fp1, fn1 = f_score(pred_list, gt_list, overlap[s])
                                tp[s] += tp1
                                fp[s] += fp1
                                fn[s] += fn1

        ACC.append(correct/total * 100)
        print("ACCURACY", correct/total * 100)

        EDIT.append(edit/7)
        print("EDIT", edit/7)

        for s in range(len(overlap)):
                precision = tp[s] / float(tp[s]+fp[s])
                recall = tp[s] / float(tp[s]+fn[s])

                f1 = 2.0 * (precision*recall) / (precision+recall)

                f1 = np.nan_to_num(f1)*100
                if s == 0:
                        F1_10.append(f1)
                if s == 1:
                        F1_25.append(f1)
                else:
                        F1_50.append(f1)
                print('F1@%0.2f: %.4f' % (overlap[s], f1))
        print()

        
print("ACC", sum(ACC)/4)
print("EDTI", sum(EDIT)/4)
print("F1@10", sum(F1_10)/4)
print("F1@25", sum(F1_25)/4)
print("F1@50", sum(F1_50)/4)



SPLIT  1
ACCURACY 46.42151481888035
EDIT 5.239414001545185
F1@0.10: 6.6775
F1@0.25: 4.5603
F1@0.50: 2.1987

SPLIT  2
ACCURACY 46.51105651105651
EDIT 5.225270686779508
F1@0.10: 7.1981
F1@0.25: 4.8762
F1@0.50: 2.9412

SPLIT  3
ACCURACY 40.661134539518436
EDIT 4.7178920495408425
F1@0.10: 6.5834
F1@0.25: 4.2376
F1@0.50: 2.1188

SPLIT  4
ACCURACY 49.63768115942029
EDIT 6.327546014651423
F1@0.10: 10.0379
F1@0.25: 6.2500
F1@0.50: 3.0303

ACC 45.80784675721889
EDTI 5.377530688129239
F1@10 7.624243388880469
F1@25 4.981007588733798
F1@50 10.196488628442252


For noun and verb only 

In [44]:
ACC = []
EDIT = []
F1_10 = []
F1_25 = []
F1_50 = []
for split in range(1, 5):

        list_vids = sorted(os.listdir('/home/ubuntu/data/hierarchical_train/results/verb/split_' + str(split)))

        print("SPLIT ", split)
        correct = 0
        total = 0
        edit = 0
        overlap = [.1, .25, .5]
        tp, fp, fn = np.zeros(3), np.zeros(3), np.zeros(3)

        for vid in list_vids:
                df = pd.read_csv("/home/ubuntu/local/Resnet50/results/weighted/verb/split"+str(split)+"/noun_vids/"+vid+".csv")
                gt_list = df["gt"].squeeze().tolist()
                pred_list = df["pred"].squeeze().tolist()
                edit += edit_score(pred_list, gt_list)

                for i in range(len(gt_list)):
                        total+=1
                        if gt_list[i]==pred_list[i]:
                                correct+=1

                for s in range(len(overlap)):
                                tp1, fp1, fn1 = f_score(pred_list, gt_list, overlap[s])
                                tp[s] += tp1
                                fp[s] += fp1
                                fn[s] += fn1
        ACC.append(correct/total * 100)
        print("ACCURACY", correct/total * 100)

        EDIT.append(edit/7)
        print("EDIT", edit/7)

        for s in range(len(overlap)):
                precision = tp[s] / float(tp[s]+fp[s])
                recall = tp[s] / float(tp[s]+fn[s])

                f1 = 2.0 * (precision*recall) / (precision+recall)

                f1 = np.nan_to_num(f1)*100
                if s == 0:
                        F1_10.append(f1)
                if s == 1:
                        F1_25.append(f1)
                else:
                        F1_50.append(f1)
                print('F1@%0.2f: %.4f' % (overlap[s], f1))
        print()

print("ACC", sum(ACC)/4)
print("EDTI", sum(EDIT)/4)
print("F1@10", sum(F1_10)/4)
print("F1@25", sum(F1_25)/4)
print("F1@50", sum(F1_50)/4)        



SPLIT  1
ACCURACY 60.834248079034026
EDIT 12.964751384578898
F1@0.10: 17.7778
F1@0.25: 13.2275
F1@0.50: 7.7249

SPLIT  2
ACCURACY 61.73218673218673
EDIT 12.835739229638307
F1@0.10: 17.6413
F1@0.25: 12.0858
F1@0.50: 7.0175

SPLIT  3
ACCURACY 57.924091960277515
EDIT 11.632537181618703
F1@0.10: 16.6501
F1@0.25: 11.2983
F1@0.50: 5.3518

SPLIT  4
ACCURACY 65.54951690821255
EDIT 15.723618010082932
F1@0.10: 21.7338
F1@0.25: 15.3846
F1@0.50: 9.1575

ACC 61.51001091992771
EDTI 13.289161451479709
F1@10 18.450768427425878
F1@25 12.999053439040921
F1@50 25.763706987560727


#### How many classes have been created ?

In [49]:
full = pd.DataFrame(columns=["noun", "verb", "gt", "prediction"])
for split in range(1,5):
    list_vids = os.listdir('/home/ubuntu/data/hierarchical_train/results/verb/split_' + str(split))
    
    for vid in list_vids:
        df = full_results(split,vid)
        full = pd.concat([full, df])
full

,noun,verb,gt,prediction
0,spoon,background,background_action,background_spoon
1,spoon,background,background_action,background_spoon
2,spoon,background,background_action,background_spoon
3,spoon,background,background_action,background_spoon
4,spoon,background,background_action,background_spoon
...,...,...,...,...
1146,action,background,background_action,background_action
1147,action,background,background_action,background_action
1148,action,background,background_action,background_action
1149,action,background,background_action,background_action


In [51]:
len(sorted(full["prediction"].unique().tolist()))

251

In [55]:
edit_score(results['pred'].tolist(), results['gt'].tolist())

15.681444991789817

# ---------------------------
### Classwise accuracy for tail and head 

# In progress ; 

In [56]:
tail = ["put_hotdog_bread", "open_mayonnaise", "put_mayonnaise", "take_jam", "open_jam", "take_ketchup", 
"close_ketchup", "put_chocolate", "take_chocolate", "close_chocolate", "close_mayonnaise", "put_jam", "stir_cup",
"put_tea"]
len(tail)

14

In [ ]:
correct_df = 

In [57]:
final_df[final_df["GT"].squeeze().str.contains(tail[2])]#contains(tail)
# final_df[final_df['frame'].squeeze().str.contains(vid)].reset_index(drop=True)

,frame,noun,gt_noun,verb,GT,pred
451,S4_Cheese_C1_0000000452.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
452,S4_Cheese_C1_0000000453.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
453,S4_Cheese_C1_0000000454.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
454,S4_Cheese_C1_0000000455.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
455,S4_Cheese_C1_0000000456.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
456,S4_Cheese_C1_0000000457.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
457,S4_Cheese_C1_0000000458.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
458,S4_Cheese_C1_0000000459.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
459,S4_Cheese_C1_0000000460.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise
460,S4_Cheese_C1_0000000461.png,mayonnaise,mayonnaise,close,put_mayonnaise,close_mayonnaise


In [20]:
final_df

,frame,noun,gt_noun,verb,GT,pred
0,S4_Cheese_C1_0000000001.png,action,action,background,background_action,background_action
1,S4_Cheese_C1_0000000002.png,action,action,background,background_action,background_action
2,S4_Cheese_C1_0000000003.png,action,action,background,background_action,background_action
3,S4_Cheese_C1_0000000004.png,action,action,background,background_action,background_action
4,S4_Cheese_C1_0000000005.png,action,action,background,background_action,background_action
...,...,...,...,...,...,...
6619,S4_Tea_C1_0000001147.png,action,action,background,background_action,background_action
6620,S4_Tea_C1_0000001148.png,action,action,background,background_action,background_action
6621,S4_Tea_C1_0000001149.png,action,action,background,background_action,background_action
6622,S4_Tea_C1_0000001150.png,action,action,background,background_action,background_action


In [60]:
## sklearn conf_mat function 
from sklearn import metrics
gt__array = final_df["GT"].values.tolist()
pred__array = final_df["pred"].values.tolist()

conf_mat_v = metrics.confusion_matrix(gt_n_array, pred_n_array)
# conf_mat = metrics.multilabel_confusion_matrix(gt_array, pred_array, labels=list(range(0,73)))
conf_mat_v

array([[1087,   54,   76, ...,   16,   28,    2],
       [ 104,   65,    4, ...,    0,    0,    0],
       [  64,   11,  122, ...,    0,    0,    0],
       ...,
       [   3,    0,    0, ...,  166,    0,    0],
       [  14,    0,    0, ...,    0,  216,    0],
       [   4,    0,    0, ...,    0,    0,  347]])

In [78]:
len(np.unique(final_df["GT"].values))

72

In [77]:
len(np.unique(final_df["pred"].values))

160

In [65]:
sum(conf_mat_v)

array([2105,  171,  229,   56,  129,   19,  116,   78,  255,  124,   46,
          0,  170,   27,   35,   34,   25,   98,   35,  123,   75,   53,
        109,  108,  266,   47,   40,  230,   89,  160,   71,  252,  261,
         57,    6,  108,  213,  250,  354])

In [1]:
 weights = [0.05209349969886704,
 5.859448301745168,
 1.1024219742974155,
 2.72445685367769,
 3.655895094251259,
 5.414426911739206,
 6.110567514677103,
 2.9704147640791474,
 0.8010107228977477,
 1.4450666419844502,
 2.263173153584112,
 1.608044082809764,
 1.0382032185131,
 3.2651887483007425,
 1.5842212075088788,
 1.5725725221595488,
 5.2163381222853324,
 3.4219178082191783,
 4.700436549751618,
 1.5276418786692758,
 1.1467552976605826,
 0.8676262191225097,
 0.8693896870475556,
 1.3161222339304532,
 1.5901105056780567,
 1.9894870978018477,
 0.7980218769167859,
 1.397842241919599,
 1.5386321080122205,
 1.1052706098899154,
 1.69067085386323,
 0.8623784798939461,
 2.1494458594341572,
 0.22301341294441987,
 0.6736058677596807,
 2.561315724714954,
 4.035280434220728,
 5.555061376979186,
 2.4724839654762847,
 2.777530688489593,
 4.550422617312737,
 6.290290088638195,
 4.035280434220728,
 4.806064337386486,
 2.72445685367769,
 3.0122515917422343,
 1.9010654490106544,
 10.184279191128507,
 1.812456466217785,
 0.7777085927770859,
 1.843705715635333,
 1.0693493150684932,
 0.9238439007071215,
 0.3400156804669295,
 0.9198703785535426,
 0.2726193282520059,
 9.505327245053273,
 0.34301501686238756,
 0.7877343020762381,
 2.5460697977821267,
 5.780266567937801,
 2.403032168693243,
 2.4166086216237135,
 1.6141121736882915,
 4.916548575027555,
 5.346746575342466,
 4.277397260273973,
 2.531004296020102,
 2.3896074079742866,
 0.40163354556563124,
 1.6643569106124407,
 2.9097940546081444,
 1.384270958017467]
print(len(weights))

73
